# Multi Location, Multiple Temporal Scales, Multiple Operations Including Transport, Mixed Integer Linear Programming Example

## [Example 4]

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming with Material and Emission Considerations Example' [Example 3]. 

We update Example 3 to now allow multiple locations and Transport options between them.


## From Example 3

In [1]:
from energia import *

m = Model('example4')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

In [2]:
m.periods

[q, y]

## Declaring Locations 

In [3]:
m.ho = Location(label='Houston')
m.sd = Location(label='San Diego')
m.ny = Location(label='New York')
# m.usa = m.ho + m.sd + m.ny

In [4]:
m.locations


[ho, sd, ny]

### The Network 

Given that we have not used a nested set of locations, a network is made:

In [5]:
m.network

ntw

This network contains the three locations at (ho, sd, ny)

In [6]:
m.network.has

(sd, ho, ny)

## General Bounds Default to Network

In [7]:
m.declare(Resource, ['power', 'wind', 'solar'])
# m.solar.consume == True
# m.wind.consume == True
m.solar.consume <= 1200
m.wind.consume <= 1200
# m.consume.show()

--- General Resource Balance for solar in (ntw, y): initializing constraint, adding consume(solar, ntw, y)
    Completed in 0.000217437744140625 seconds
--- Binding consume in domain (solar, ntw, y)
    Completed in 9.679794311523438e-05 seconds
--- General Resource Balance for wind in (ntw, y): initializing constraint, adding consume(wind, ntw, y)
    Completed in 0.00013756752014160156 seconds
--- Binding consume in domain (wind, ntw, y)
    Completed in 9.1552734375e-05 seconds


## Location-specific Bounds 

Spatial specificity can be provided along with temporal fidelity. 
In the example below, the temporal fidelity is implied since the data has 4 samples.

Note that an individual general resource balance is generated for power at each location

In [8]:
m.power.release(m.ho).prep(30) >= [0.6, 0.7, 0.8, 0.3]
m.power.release(m.sd).prep(100) >= [0.4, 0.9, 0.7, 0.6]
m.power.release(m.ny).prep(180) >= [0.2, 0.5, 0.7, 0.5]
m.release.show()

--- General Resource Balance for power in (ho, q): initializing constraint, adding release(power, ho, q)
    Completed in 0.00023102760314941406 seconds
--- Binding release in domain (power, ho, q)
    Completed in 0.00025200843811035156 seconds
--- General Resource Balance for power in (sd, q): initializing constraint, adding release(power, sd, q)
    Completed in 0.0004909038543701172 seconds
--- Binding release in domain (power, sd, q)
    Completed in 0.0002429485321044922 seconds
--- General Resource Balance for power in (ny, q): initializing constraint, adding release(power, ny, q)
    Completed in 0.00018095970153808594 seconds
--- Binding release in domain (power, ny, q)
    Completed in 0.0002181529998779297 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## For All Locations 

Generally, in Energia to repeat a constraint over a set, use .forall()

If a list is provided, the bounds are iterated over the list [see LB cons]
else the bounds are repeated [see UB cons]

In [9]:
m.wf = Process(m.power)
m.wf(m.power) == -1 * m.wind
[10, 7, 20] <= m.wf.capacity.x.forall(m.network.has)
m.wf.capacity.x.forall(m.network.has) <= 100
m.wf.show()

--- Binding capacity in domain (wf, sd, y)
    Completed in 0.00018978118896484375 seconds
--- Binding capacity in domain (wf, ho, y)
    Completed in 0.000133514404296875 seconds
--- Binding capacity in domain (wf, ny, y)
    Completed in 0.0002613067626953125 seconds
--- Binding capacity in domain (wf, sd, y)
    Completed in 0.00020837783813476562 seconds
--- Binding capacity in domain (wf, ho, y)
    Completed in 0.00020647048950195312 seconds
--- Binding capacity in domain (wf, ny, y)
    Completed in 0.0001888275146484375 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Bounds can also be provided individually

In [10]:
m.wf.operate.prep(norm=True)(m.ho) <= [0.9, 0.8, 0.5, 0.7]
m.wf.operate.prep(norm=True)(m.sd) <= [0.8, 0.7, 0.6, 0.5]
m.wf.operate.prep(norm=True)(m.ny) <= [0.6, 0.5, 0.4, 0.3]
m.operate.show(True)
# m.wf.operate.prep(norm = True).forall(m.network.has) <= [[0.9, 0.8, 0.5, 0.7], [0.8, 0.7, 0.6, 0.5], [0.6, 0.5, 0.4, 0.3]]

--- Binding operate in domain (wf, ho, q)
    Completed in 0.00046825408935546875 seconds
--- Binding operate in domain (wf, sd, q)
    Completed in 0.00022482872009277344 seconds
--- Binding operate in domain (wf, ny, q)
    Completed in 0.0002002716064453125 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Calculations can also be provided for all locations as a list or repeated

In [11]:
m.wf.capacity[m.usd.spend].forall(m.network.has) == [90000, 70000, 60000]
m.wf.operate[m.usd.spend] == 49

In the case where location specific data is not provided, the parameter is assumed tuo apply to the network

In [12]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar

In [13]:
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True).forall(m.network.has) <= [
    [0.6, 0.8, 0.9, 0.7],
    [0.5, 0.7, 0.6, 0.5],
    [0.4, 0.6, 0.5, 0.4],
]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

m.gwp = Environ()

m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000

m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- Binding capacity in domain (pv, ntw, y)
    Completed in 0.00024771690368652344 seconds
--- Binding capacity in domain (pv, ntw, y)
    Completed in 0.0001819133758544922 seconds
--- Binding operate in domain (pv, sd, q)
--- Aspect (capacity) not defined at sd, a variable will be created assuming y as the temporal index
--- Creating map to (pv, ntw, y). Mapping capacity: from (pv, sd, y) to (pv, ntw, y)
    Completed in 0.0001163482666015625 seconds
    Completed in 0.0005738735198974609 seconds
--- Binding operate in domain (pv, ho, q)
--- Aspect (capacity) not defined at ho, a variable will be created assuming y as the temporal index
--- Mapping capacity: from (pv, ho, y) to (pv, ntw, y)
    Completed in 0.00010251998901367188 seconds
    Completed in 0.0006687641143798828 seconds
--- Binding operate in domain (pv, ny, q)
--- Aspect (capacity) not defined at ny, a variable will be created assuming y as the temporal index
--- Mapping capacity: from (pv, ny, y) to (pv, ntw, y)
    

## Linkages 

A grid of valid linkages can be created from sources to sinks.

Linkages are always one directional, stating bi, created a link from source to sink and the other way round 

If multiple links exist between two locations with different distances, it is necessary to create named links. 
Again, it needs to be stated whether the links are bi directional 

In [14]:
m.Link(source=m.ho, sink=m.sd, dist=2000, bi=True)
# m.Link(source = m.sd, sink=m.ny, dist = 1500)
m.road = Linkage(source=m.ho, sink=m.ny, dist=3000, bi=True)
# m.rail = Link(source = m.ho, sink=m.ny, dist = 5000, bi = True)
m.linkages

[ho-sd, sd-ho, road, -road]

In [15]:
m.ho.links(m.sd)

ho is source and sd is sink in ho-sd
sd is source and ho is sink in sd-ho


[ho-sd, sd-ho]

## Transits

Declare some transport options between across the linkages 


In [16]:
m.show()

# Mathematical Program for Program(example4)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Mapping Constraint Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
m.declare(Resource, ['h2o', 'co2'])
m.grid = Transport()
m.grid(m.power) == - m.h2o + m.co2 
m.grid.capacity.x.forall([m.ho - m.sd, m.road]) <= 100
m.grid.operate.prep(norm=True).forall([m.ho - m.sd, m.road]) <= [
    [0.9, 0.8, 0.5, 0.7],
    [0.8, 0.7, 0.6, 0.9],
]
m.grid.operate[m.usd.spend] == 2000
m.grid.capacity[m.usd.spend] == 1000

--- Binding capacity in domain (grid, ho-sd, y)
    Completed in 0.0004994869232177734 seconds
--- Binding capacity in domain (grid, road, y)
    Completed in 0.0001666545867919922 seconds
--- Binding operate in domain (grid, ho-sd, q)
    Completed in 0.00021958351135253906 seconds
--- Binding operate in domain (grid, road, q)
    Completed in 0.0001952648162841797 seconds


In [18]:
m.ho.locate(m.pv, m.wf, m.lii)
m.sd.locate(m.pv, m.wf, m.lii)
m.ny.locate(m.pv, m.wf, m.lii)

--- Assuming  pv capacity is unbounded in (ho, y)
--- General Resource Balance for power in (ho, q): adding produce(power, ho, q, operate, pv)
    Completed in 0.00017404556274414062 seconds


KeyError: ho

In [19]:
m.maps 

{capacity: {(pv, ntw, y): [(pv, sd, y), (pv, ho, y), (pv, ny, y)],
  (wf, ntw, y): [(wf, ho, y), (wf, sd, y), (wf, ny, y)]}}

In [21]:
m.dispositions

{consume: {solar: {ntw: {y: {}}},
  wind: {ntw: {y: {}}},
  cement: {ntw: {y: {}}}},
 release: {power: {ho: {q: {}}, sd: {q: {}}, ny: {q: {}}}},
 capacity: {wf: {sd: {y: {}}, ho: {y: {}}, ny: {y: {}}, ntw: {y: {}}},
  pv: {ntw: {y: {}}, sd: {y: {}}, ho: {y: {}}, ny: {y: {}}},
  lii.charge: {ntw: {y: {}}},
  lii.discharge: {ntw: {y: {}}},
  grid: {ho-sd: {y: {}}, road: {y: {}}, ntw: {y: {}}}},
 operate: {wf: {ho: {q: {}}, sd: {q: {}}, ny: {q: {}}, ntw: {y: {}}},
  pv: {sd: {q: {}}, ho: {q: {}}, ny: {q: {}}, ntw: {y: {}}},
  lii.charge: {ntw: {y: {}}},
  lii.discharge: {ntw: {y: {}}},
  grid: {ho-sd: {q: {}}, road: {q: {}}, ntw: {y: {}}}},
 spend: {usd: {sd: {y: {capacity: {wf: {}}}},
   ho: {y: {capacity: {wf: {}}}},
   ny: {y: {capacity: {wf: {}}}},
   ntw: {y: {operate: {wf: {}, pv: {}, grid: {}},
     capacity: {pv: {}, grid: {}},
     invcapacity: {power.lii: {}},
     inventory: {power.lii: {}},
     consume: {cement: {}}}}}},
 invcapacity: {power.lii: {ntw: {y: {}}}},
 inventory: 

In [20]:
m.show()

# Mathematical Program for Program(example4)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Mapping Constraint Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.grb

{solar: {ntw: {y: [consume(solar, ntw, y)], q: []}},
 wind: {ntw: {y: [consume(wind, ntw, y)]}},
 power: {ho: {q: [release(power, ho, q), produce(power, ho, q, operate, pv)]},
  sd: {q: [release(power, sd, q)]},
  ny: {q: [release(power, ny, q)]}},
 usd: {sd: {y: []}, ho: {y: []}, ny: {y: []}, ntw: {y: []}, None: {y: []}},
 power.lii: {ntw: {y: [inventory(power.lii, ntw, y)]}},
 cement: {ntw: {y: [consume(cement, ntw, y),
    use(cement, ntw, y, capacity, wf),
    use(cement, ntw, y, capacity, pv),
    use(cement, ntw, y, invcapacity, power.lii)]}}}

In [ ]:
m.solar.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.usd.spend.opt()

bbbb spend (usd, ntw, y) False
--- Mapping spend: from (usd, capacity, wf, sd, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, wf, ho, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, wf, ny, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, wf, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, pv, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, pv, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, lii, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, lii, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, consume, cement, ntw, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, ho-sd, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, sd-ho, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, road, y) to (usd, ntw, y)
--- Mapping spend: from (usd, operate, grid, -road, y) to (usd, ntw, y)
--- Mapping spend: from (usd, capacity, grid, ntw, y) to (usd,

Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file example4.mps
Reading time = 0.00 seconds
EXAMPLE4: 233 rows, 241 columns, 572 nonzeros
--- Optimizing example4 using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 233 rows, 241 columns and 572 nonzeros
Model fingerprint: 0x30d71bce
Variable types: 234 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+06]
Presolve removed 166 rows and 170 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible
Best

In [ ]:
m.sol()

'Use .opt() to generate solution'

In [ ]:
m.export.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.spend.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>